In [ ]:
import pandas as pd
import geopy.distance
from math import sin, cos, sqrt, atan2, radians 

In [ ]:
df = pd.read_csv("address_with_coords.csv").dropna()
df["lon"] = df.point.apply(lambda x: float(x.split()[0]))
df["lat"] = df.point.apply(lambda x: float(x.split()[1]))
df = df[(df["lat"]>55.582238) & (df["lat"]<55.905471) & (df["lon"]>37.372692) & (df["lon"]<37.851569)]

In [ ]:
#clear from non-Moscow objects

#Moscow shape:
#lat: 55.582238 .. 55.905471
#lon: 37.372692 .. 37.851569
#38 х 33 км

fishnet_df = pd.read_csv("fishnet.csv")
fishnet_df.columns = ["lon", "lat", "cell_id", "cell_area", "cell_area2"]
fishnet_df = fishnet_df[["lon", "lat", "cell_id"]]
fishnet_msc_df = fishnet_df[(fishnet_df["lat"]>55.582238) & (fishnet_df["lat"]<55.905471) & (fishnet_df["lon"]>37.372692) & (fishnet_df["lon"]<37.851569)]

In [ ]:
#coords_1 = (52.2296756, 21.0122287)
#coords_2 = (52.406374, 16.9251681)

#print geopy.distance.vincenty(coords_1, coords_2).km


def find_closest1(lon1, lat1):
  dists = []

  for row in fishnet_msc_df.iterrows():
    coords_1 = (row[1].lat, row[1].lon)
    coords_2 = (lat1, lon1)
    dists.append(geopy.distance.vincenty(coords_1, coords_2).km)
  return fishnet_msc_df.cell_id.tolist()[dists.index(min(dists))]


def find_closest2(lon1, lat1):
  dists = []
  R = 6373.0
  lat2 = radians(lat1)
  lon2 = radians(lon1)
  
  for row in fishnet_msc_df.iterrows():

    lat1 = radians(row[1].lat)
    lon1 = radians(row[1].lon)
 
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    dists.append(distance)
  
  return fishnet_msc_df.cell_id.tolist()[dists.index(min(dists))]

In [ ]:
ids_ = []
for row in df.iterrows():
  point = row[1].point
  lon = float(point.split()[0])
  lat = float(point.split()[1])

  id_ = find_closest1(lon, lat)
  ids_.append(id_)